In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

# 检查GPU是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载模型和分词器
tokenizer = AutoTokenizer.from_pretrained("hw2942/bert-base-chinese-finetuning-financial-news-sentiment-v2")
model = AutoModelForSequenceClassification.from_pretrained("hw2942/bert-base-chinese-finetuning-financial-news-sentiment-v2")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm

news_filtered = pd.read_csv('/content/drive/MyDrive/data/News_filtered.csv')

In [ ]:
# 将模型移到GPU
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# 准备数据
texts = []
for _, row in news_filtered.iterrows():
    texts.append(row['Title'] + ' ' + row['NewsContent'])

In [ ]:
import gc

chunk_size = 10000  # 每次处理个文本

from torch.utils.data import Dataset
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

predictions = []
# 分批处理文本
for i in tqdm(range(0, len(texts), chunk_size)):
    chunk = texts[i:i + chunk_size]
    # 使用tokenizer处理当前批次
    encoded_chunk = tokenizer(chunk, padding=True, truncation=True, max_length=512, return_tensors="pt")
    # 处理编码后的数据，例如保存到文件或用于模型训练
    dataset = TextDataset(encoded_chunk)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    # 创建DataLoader
    data_loader = DataLoader(dataset, batch_size=256, collate_fn=data_collator)
    # 批量预测
    model.eval()
    with torch.no_grad():
        for batch in tqdm(data_loader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            predictions.extend(torch.argmax(outputs.logits, dim=-1).tolist())
    # 清理内存
    del encoded_chunk
    gc.collect()


<ipython-input-11-c530ff85105e>:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, len(texts), chunk_size)):


  0%|          | 0/100 [00:00<?, ?it/s]

<ipython-input-11-c530ff85105e>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm(data_loader):


  0%|          | 0/40 [00:00<?, ?it/s]

<ipython-input-11-c530ff85105e>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


...

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

In [ ]:
pred_sr = pd.Series(predictions)

In [ ]:
pred_sr.describe()

count    995457.000000
mean          0.958553
std           0.629822
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           2.000000
dtype: float64

In [ ]:
pred_sr.value_counts()

1    598874
0    218921
2    177662
dtype: int64

In [ ]:
sentiment_output = pd.DataFrame()
sentiment_output['NewsID'] = news_filtered['NewsID']
sentiment_output['Sentiment'] = pred_sr
sentiment_output.to_csv('/content/drive/MyDrive/data/sentiment_output.csv', index=False)

In [ ]:
#